## Setup

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

Let's see what the database looks like

In [2]:
%%sql
SELECT * FROM facts limit 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [3]:
%%sql
SELECT * FROM sqlite_master WHERE type='table'

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Population and Population Growth
Let's find the minimums and maximums for populations and populations growth

In [10]:
%%sql
SELECT MIN(population) AS 'Minimum Population', MAX(population) AS 'Maximum Population', 
       MIN(population_growth) AS 'Minimum Population Growth', MAX(population_growth) AS 'Maximum Population Growth'
  FROM facts

 * sqlite:///factbook.db
Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,7256490011,0.0,4.02


Looks like there are some weird data. Let's check out what it is.

In [27]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population) 
                      FROM facts) 
    OR population == (SELECT MAX(population) 
                      FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


## Average population
Let's find out the countries where the population is bigger than the average and the area is lower than the average.

In [25]:
%%sql
SELECT name, ROUND(AVG(population),0) AS 'Average Population', ROUND(AVG(area),0) AS 'Average Area'
  FROM facts
 WHERE population > (SELECT AVG(population) 
                    FROM facts) 
   AND area < (SELECT AVG(area) 
              FROM facts)
 GROUP BY NAME

 * sqlite:///factbook.db
Done.


name,Average Population,Average Area
Bangladesh,168957745.0,148460.0
Germany,80854408.0,357022.0
Japan,126919659.0,377915.0
Philippines,100998376.0,300000.0
Thailand,67976405.0,513120.0
United Kingdom,64088222.0,243610.0
Vietnam,94348835.0,331210.0


Let's find out which countries have the highest ratios of water to land.

In [36]:
%%sql
SELECT name, area_land, area_water, (area_water / area_land) AS 'ratio'
FROM facts
WHERE ratio > (SELECT AVG(area_water) / AVG(area_land) 
               FROM facts)
GROUP BY name

 * sqlite:///factbook.db
Done.


name,area_land,area_water,ratio
British Indian Ocean Territory,60,54340,905
Virgin Islands,346,1564,4
